### Install libraries

In [ ]:
%pip install rasterio geopandas shapely matplotlib folium pandas

### Load and Visualize DEM Data
Use rasterio to read the DEM file and matplotlib to visualize the elevation.

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

# Load DEM file
dem_path = "data/s24_w047_1arc_v3.tif"
with rasterio.open(dem_path) as dem:
    elevation = dem.read(1)  # Read elevation data (band 1)
    extent = [dem.bounds.left, dem.bounds.right, dem.bounds.bottom, dem.bounds.top]

# Plot the DEM
plt.figure(figsize=(10, 6))
plt.imshow(elevation, cmap="terrain", extent=extent)
plt.colorbar(label="Elevation (m)")
plt.title("Digital Elevation Model (DEM) of São Paulo")
plt.show()


### Identify Low-Lying Areas (Flood-Prone Zones)
Identify low-lying areas in the DEM data that are prone to flooding. These areas are typically flat and have low elevation.

In [ ]:
# Define a threshold for low elevation (e.g., below 50 meters)
low_elevation_threshold = 50
low_elevation = elevation < low_elevation_threshold

# Plot low-lying areas
plt.figure(figsize=(10, 6))
plt.imshow(low_elevation, cmap="Reds", extent=extent)
plt.title("Potential Flood Zones (Low Elevation)")
plt.show()


In [ ]:
from scipy.ndimage import gaussian_gradient_magnitude

# Calculate the gradient magnitude (slope) of the elevation data
slope = gaussian_gradient_magnitude(elevation.astype(float), sigma=1)

# Define a threshold for flat areas (e.g., slope below a certain value)
flat_threshold = 2  # Adjust this value as needed
flat_areas = slope < flat_threshold

# Combine low elevation and flatness to identify flood-prone zones
flood_prone_zones = low_elevation & flat_areas
# print(set(flood_prone_zones.flatten()))

# Plot flood-prone zones
plt.figure(figsize=(10, 6))
plt.imshow(flood_prone_zones, cmap="Reds", extent=extent)
plt.title("Flood-Prone Zones (Low Elevation and Flat Areas)")
plt.show()

### Load and Plot River & Waterbody Data
Use geopandas to read the river and waterbody shapefiles and matplotlib to visualize them.

In [ ]:
import matplotlib.patches as mpatches
import geopandas as gpd

# Load river and waterbody data
rivers_path = "data/sao_paulo_waterways.geojson"
waterbodies_path = "data/sao_paulo_water.geojson"

rivers = gpd.read_file(rivers_path)
waterbodies = gpd.read_file(waterbodies_path)

# Plot rivers and waterbodies over the DEM
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(flood_prone_zones, cmap="Reds", extent=extent)

rivers.plot(ax=ax, color="blue", linewidth=1)
waterbodies.plot(ax=ax, color="cyan", alpha=0.5)

# Manually create legend patches
river_patch = mpatches.Patch(color="blue", label="Rivers")
waterbody_patch = mpatches.Patch(color="cyan", label="Waterbodies")
plt.legend(handles=[river_patch, waterbody_patch], loc="upper right")
plt.title("Flood Risk Areas with Rivers and Waterbodies")
plt.show()

### Load and Analyze Rainfall Data
Use pandas to read the rainfall data and matplotlib to visualize it.

In [ ]:
import pandas as pd

# Load rainfall data
rainfall_path = "data/sao_paulo_rain_2021-2025.csv"
rainfall_data = pd.read_csv(rainfall_path)

# Display first few rows
print(rainfall_data.head())

# Keep only rows where NAME is "SAO PAULO AEROPORT, BR"
rainfall_data = rainfall_data[rainfall_data["NAME"] == "SAO PAULO AEROPORT, BR"]

# Drop unnecessary columns
rainfall_data = rainfall_data[["DATE", "PRCP", "LATITUDE", "LONGITUDE"]]  # Keeping only date and precipitation

# Convert DATE column to datetime
rainfall_data["DATE"] = pd.to_datetime(rainfall_data["DATE"])

# Set DATE as the index
rainfall_data.set_index("DATE", inplace=True)

# Check for missing values
# print(rainfall_data.isnull().sum())

# Fill missing precipitation values with 0 (assuming no rainfall recorded)
rainfall_data.fillna({"PRCP": 0}, inplace=True)

print(rainfall_data.head())


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Plot precipitation data
plt.figure(figsize=(12, 6))
plt.plot(rainfall_data.index, rainfall_data["PRCP"], color="blue", label="Precipitation (mm)")

# Format x-axis labels
plt.gca().xaxis.set_major_locator(mdates.YearLocator())  # Show only years
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y"))  # Format as YYYY
plt.gca().xaxis.set_minor_locator(mdates.MonthLocator())  # Optional: add minor ticks for months

plt.xlabel("Year")
plt.ylabel("Rainfall (mm)")
plt.title("Rainfall in São Paulo (2021-2025)")
plt.legend()
plt.grid()
plt.xticks(rotation=45)
plt.show()

### Combine Data for Flood Risk Analysis
Combine the DEM, river, waterbody, and rainfall data to identify areas at risk of flooding.

**Step 1: Convert Data to Geospatial Format**  
Convert the DEM, rivers, and rainfall data into a compatible geospatial format.

Convert DEM to a Binary Flood Risk Layer   
Classify areas as flood-prone if their elevation is below a threshold.

In [ ]:
import numpy as np

# Define flood-prone elevation threshold
low_elevation_threshold = 50  # Adjust as needed

# Create a binary flood risk layer (1 = flood-prone, 0 = safe)
flood_risk = (elevation < low_elevation_threshold).astype(int)

# Plot
plt.figure(figsize=(10, 6))
plt.imshow(flood_risk, cmap="Reds", extent=extent)
plt.title("Flood-Prone Areas (Low Elevation)")
plt.show()


**Step 2: Buffer River Data (Proximity to Water)**  
Flooding risk is higher near rivers and lakes. Create a buffer zone around them.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Step 1: Reproject to Projected CRS (UTM zone 23S)
rivers_projected = rivers.to_crs("EPSG:32723")
waterbodies_projected = waterbodies.to_crs("EPSG:32723")

# Step 2: Create buffers in the projected CRS
buffer_distance_meters = 500  # Buffer size in meters

rivers_buffered = rivers_projected.copy()
rivers_buffered["geometry"] = rivers_projected.geometry.buffer(buffer_distance_meters)

waterbodies_buffered = waterbodies_projected.copy()
waterbodies_buffered["geometry"] = waterbodies_projected.geometry.buffer(buffer_distance_meters)

# Step 3: Plot with manual legend handles
fig, ax = plt.subplots(figsize=(10, 6))
rivers_buffered.plot(ax=ax, color="blue", alpha=0.5)
waterbodies_buffered.plot(ax=ax, color="cyan", alpha=0.5)

# Manually add legend handles
river_patch = mpatches.Patch(color="blue", label="Buffered Rivers (500m)")
waterbody_patch = mpatches.Patch(color="cyan", label="Buffered Waterbodies (500m)")

plt.legend(handles=[river_patch, waterbody_patch], loc="upper right")
plt.title("Buffered Water Sources in Projected CRS with Legend Fix")
plt.show()


**Step 3: Identify High Rainfall Zones**  
Filter days with extreme rainfall and create a spatial join with the DEM.  
(For simplicity, we will use a threshold value to identify high rainfall days. Additionally, current data only contains rainfall from one station, at Sao Paulo Airport. So only one point will be used to represent the entire city.)

In [ ]:
# Define extreme rainfall threshold
heavy_rain_threshold = 50  # Adjust as needed

# Filter high rainfall days
heavy_rain_days = rainfall_data[rainfall_data["PRCP"] > heavy_rain_threshold]

# Convert to GeoDataFrame
rainfall_gdf = gpd.GeoDataFrame(
    heavy_rain_days.reset_index(),
    geometry=gpd.points_from_xy(heavy_rain_days["LONGITUDE"], heavy_rain_days["LATITUDE"]),
    crs="EPSG:4326"
)

# Plot rainfall stations with high precipitation
fig, ax = plt.subplots(figsize=(10, 6))
rivers.plot(ax=ax, color="blue", linewidth=0.5)
rainfall_gdf.plot(ax=ax, color="red", markersize=50, alpha=0.6, label="Heavy Rainfall Stations")
plt.legend()
plt.title("Heavy Rainfall Locations (>50mm)")
plt.show()


**Step 4: Combine All Risk Factors**  
Merge low elevation, proximity to water, and high rainfall.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Step 1: Define flood zones by merging buffered rivers and waterbodies
flood_zones = gpd.overlay(rivers_buffered, waterbodies_buffered, how="union")

# Ensure rainfall data is in the same CRS before the join
rainfall_gdf = rainfall_gdf.to_crs(flood_zones.crs)

# Perform the spatial join
rainfall_flood_zone = gpd.sjoin(rainfall_gdf, flood_zones, how="inner", predicate="intersects")

# Reproject everything back to EPSG:4326 for map coordinates
flood_zones_map = flood_zones.to_crs("EPSG:4326")
rainfall_flood_zone_map = rainfall_flood_zone.to_crs("EPSG:4326")

# Repair invalid geometries
flood_zones_map["geometry"] = flood_zones_map["geometry"].buffer(0)
rainfall_flood_zone_map["geometry"] = rainfall_flood_zone_map["geometry"].buffer(0)

# Check for empty GeoDataFrames before plotting
if not flood_zones_map.empty: #and not rainfall_flood_zone_map.empty:
    fig, ax = plt.subplots(figsize=(10, 6))
    flood_zones_map.plot(ax=ax, color="lightblue", alpha=0.5, label="Flood Zones")
    #rainfall_flood_zone_map.plot(ax=ax, color="red", markersize=50, label="High Rainfall Areas")
    
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.legend()
    plt.title("Final Flood Risk Map with Map Coordinates (EPSG:4326)")
    plt.show()
else:
    print("No valid geometries to plot.")

